# Final Project - IBM Applied Data Science Capstone 
#### By Anzar Syed

## **For this project I analyse the rapidly growing market for healthy food ventures in San Francisco, one of the biggest consumers of health food products in the US to make sense of the competition in this market and gain actionable insights such as market concentration, geographic saturation and other similar trends.**
# ______________

###  Data Scources:

#### Foursquare API  - to explore the features of neighbourhoods in San Francisco and to acquire data of various venue categories related to the health food market, namely Organic Grocery, Farmers Market, and Vegetarian / Vegan Restaurant
#### The table on: http://www.healthysf.org/bdi/outcomes/zipmap.htm for data of San Francisco neighbourhoods and their corresponding Zip codes
#### The python database uszipcode to get the longitude and latitude values corresponding to each zip code 
#### The geopy geocoder class Nominatum will also be used for geocoding certain fields of data into longitude and latitude values
    


# _________________________________________
#### As i go long I will use markdowns to explain my code as well as what the output means and, where appropriate, a brief analysis of the implications of the output
#    ______________

#### Downloading all the dependencies needed

In [4]:
import requests
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
from IPython.display import display_html
import numpy as np
import random 

!pip install uszipcode
from uszipcode import SearchEngine

from bs4 import BeautifulSoup
import json

!pip install geocoder
from geopy.geocoders import Nominatim

from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: done

# All requested packages already installed.



#### Scraping the following Wikipedia page: http://www.healthysf.org/bdi/outcomes/zipmap.htm using the Beautiful Soup Python package for parsing HTML and XML documents and obtaining the data for neighborhoods in San Francisco and their corresponding Zip Codes in the table as a dataframe


In [5]:
response = requests.get("http://www.healthysf.org/bdi/outcomes/zipmap.htm")
soup = BeautifulSoup(response.text, "lxml")
table = soup.find_all("table")
df = pd.read_html(str(table))
df = pd.DataFrame(df[4])


In [6]:

df.columns = df.iloc[0]
df = df.iloc[1:-1, :-1]
sf_data = df
sf_data.head()

,Zip Code,Neighborhood
1,94102,Hayes Valley/Tenderloin/North of Market
2,94103,South of Market
3,94107,Potrero Hill
4,94108,Chinatown
5,94109,Polk/Russian Hill (Nob Hill)


#### In order to utilize Foursquare location data, we need to get the latitude and the longitude coordinates of each neighborhood. I will use the Python database uszipcode and its function SearchEngine to acquire this


In [7]:
search = SearchEngine(simple_zipcode=True)

latitude = []
longitude = []

for index, row in df.iterrows():
    zipcode = search.by_zipcode(row["Zip Code"]).to_dict()
    latitude.append(zipcode.get("lat"))
    longitude.append(zipcode.get("lng"))

sf_data["Latitude"] = latitude
sf_data["Longitude"] = longitude

sf_data.head()

,Zip Code,Neighborhood,Latitude,Longitude
1,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420
2,94103,South of Market,37.780,-122.410
3,94107,Potrero Hill,37.770,-122.390
4,94108,Chinatown,37.791,-122.409
5,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420


#### The geopy geocoder class Nominatum will also be used for geocoding the address San Francisco into longitude and latitude values

In [8]:
address = 'San Francisco'

geolocator = Nominatim(user_agent = "san_francisco_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of San Francisco are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of San Francisco are 37.7790262, -122.4199061.


#### I now use the Folium Library to visualize the geo-spatial data of neighbourhoods of San Francisco on a Map

In [9]:

map_sf = folium.Map(location = [latitude, longitude], zoom_start=12)

# add markers to map
for lat, lng, neighborhood in zip(sf_data['Latitude'], sf_data['Longitude'], sf_data['Neighborhood']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html = True)
    folium.CircleMarker(
        [lat, lng],
        radius = 5,
        popup = label,
        color = 'blue',
        fill = True,
        fill_color = '#3186cc',
        fill_opacity = 0.7,
        parse_html = False).add_to(map_sf)  
    
map_sf

#### Connecting with the Foursquare API

In [11]:
CLIENT_ID = 'KMGBLPZIU11KFN5LTRQY0OLTBIZEJV04X0ISMOVRODAYY0WG' 
CLIENT_SECRET = 'N00J5YJY3SAMKCFCSL1MGQKL4BUJFVWIPYD1BXNRCQ1BFRDB' 
VERSION = '20180605'

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: KMGBLPZIU11KFN5LTRQY0OLTBIZEJV04X0ISMOVRODAYY0WG
CLIENT_SECRET:N00J5YJY3SAMKCFCSL1MGQKL4BUJFVWIPYD1BXNRCQ1BFRDB


#### Getting data of venues in the neighbourhoods from the Foursquare API and organising it

In [12]:
LIMIT=200
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        results = requests.get(url).json()["response"]['groups'][0]['items']
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    return(nearby_venues)


In [13]:
sf_venues = getNearbyVenues(names = sf_data['Neighborhood'],
                                   latitudes = sf_data['Latitude'],
                                   longitudes = sf_data['Longitude']
                                  )
                                  
sf_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Herbst Theater,37.779548,-122.420953,Concert Hall
1,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,War Memorial Opera House,37.778601,-122.420816,Opera House
2,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,San Francisco Ballet,37.778580,-122.420798,Dance Studio
3,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,Louise M. Davies Symphony Hall,37.777976,-122.420157,Concert Hall
4,Hayes Valley/Tenderloin/North of Market,37.78,-122.42,The Nutcracker,37.778569,-122.420800,Performing Arts Venue


#### One-hot encoding: turning the qualitative data into quantitative data for further analysis

In [16]:
# one hot encoding
sf_onehot = pd.get_dummies(sf_venues[['Venue Category']], prefix = "", prefix_sep = "")

# add neighborhood column back to dataframe
sf_onehot['Neighborhood'] = sf_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [sf_onehot.columns[-1]] + list(sf_onehot.columns[:-1])
sf_onehot = sf_onehot[fixed_columns]

sf_onehot.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Alternative Healer,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Bike Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Café,Camera Store,Candy Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,Hayes Valley/Tenderloin/North of Market,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

#### Grouping the data by neighbourhood

In [17]:
sf_grouped = sf_onehot.groupby('Neighborhood').mean().reset_index()
sf_grouped.head()

,Neighborhood,ATM,Accessories Store,Adult Boutique,Afghan Restaurant,Alternative Healer,American Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,BBQ Joint,Bakery,Bank,Bar,Baseball Field,Basketball Court,Basketball Stadium,Bed & Breakfast,Beer Bar,Bike Shop,Bookstore,Boutique,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Building,Burger Joint,Burmese Restaurant,Burrito Place,Bus Line,Bus Station,Café,Camera Store,Candy Store,Cantonese Restaurant,Chinese Restaurant,Chocolate Shop,Church,Clothing Store,Cocktail Bar,Coffee Shop,College Quad,Comic Shop,Concert Hall,Cosmetics Shop,Coworking Space,Credit Union,Creperie,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Dive Bar,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Electronics Store,English Restaurant,Ethiopian Restaurant,Event Space,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market,Flower Shop,Fondue Restaurant,Food,Food & Drink Shop,Food Truck,Fountain,French Restaurant,Fried Chicken Joint,Furniture / Home Store,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,Gift Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Harbor / Marina,Hardware Store,Herbs & Spices Store,Hill,Historic Site,History Museum,Hot Dog Joint,Hotel,Hotel Bar,Hotpot Restaurant,Hunan Restaurant,Hungarian Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Italian Restaurant,Japanese Curry Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Kitchen Supply Store,Korean Restaurant,Latin American Restaurant,Library,Light Rail Station,Liquor Store,Lounge,Luggage Store,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Mexican Restaurant,Middle Eastern Restaurant,Mini Golf,Mobile Phone Shop,Monument / Landmark,Moroccan Restaurant,Motel,Motorcycle Shop,Movie Theater,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Noodle House,North Indian Restaurant,Opera House,Optical Shop,Organic Grocery,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Pet Store,Pharmacy,Pier,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Pool,Pool Hall,Pop-Up Shop,Pub,Public Art,Ramen Restaurant,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Scandinavian Restaurant,Scenic Lookout,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shipping Store,Shoe Repair,Shoe Store,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Steakhouse,Street Food Gathering,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Wine Bar,Wine Shop,Yoga Studio
0,Bayview-Hunters Point,0.0,0.00,0.000000,0.0,0.0,0.000000,0.0,0.00,0.0,0.00,0.0,0.0,0.000000,0.00,0.00,0.0,0.000000,0.0,0.00,0.00,0.0,0.000000,0.000000,0.0,0.0,0.00,0.0,0.00,0.00,0.00,0.0,0.000000,0.0,0.0,0.000000,0.00,0.00,0.0,0.00,0.00,0.00,0.000000,0.00,0.200000,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.00,0.000000,0.00,0.0,0.000000,0.0,0.00,0.000000,0.000000,0.00,0.0,0.0,0.00,0.0,0.0,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.20,0.00,0.00,0.00,0.0,0.00,0.00,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.00,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.00,0.00,0.000000,0.00,0.00,0.000000,0.0,0.0,0.00,0.00,0.0,0.0,0.0,0.00,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.00,0.00,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.2,0.0,0.0,0.0,0.00,0.000000,0.000000,0.0,0.0,0.00,0.0000

#### Seeing the top 10 most common venue categories for each neighbourhood

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending = False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [19]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind + 1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind + 1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = sf_grouped['Neighborhood']

for ind in np.arange(sf_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(sf_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bayview-Hunters Point,Food & Drink Shop,Motorcycle Shop,Park,Coffee Shop,Restaurant,Ethiopian Restaurant,Food,Fondue Restaurant,Flower Shop,Fish Market
1,Castro/Noe Valley,Gay Bar,Park,Yoga Studio,Grocery Store,Thai Restaurant,Clothing Store,Coffee Shop,Playground,Wine Bar,Pet Store
2,Chinatown,Hotel,Coffee Shop,American Restaurant,Boutique,Men's Store,Cocktail Bar,Church,Jewelry Store,Spa,Café
3,Haight-Ashbury,Coffee Shop,Tennis Court,Grocery Store,Yoga Studio,Dog Run,Boutique,Bookstore,Restaurant,Gastropub,Comic Shop
4,Hayes Valley/Tenderloin/North of Market,Coffee Shop,Cocktail Bar,Performing Arts Venue,Vietnamese Restaurant,Café,French Restaurant,Theater,Beer Bar,Mexican Restaurant,Southern / Soul Food Restaurant


#### Clustering the data into five distinct clusters

In [20]:
from sklearn.cluster import KMeans
# set number of clusters
kclusters = 5
sf_grouped_clustering = sf_grouped.drop('Neighborhood', 1)
# run k-means clustering
kmeans = KMeans(n_clusters = kclusters, random_state = 0).fit(sf_grouped_clustering)
# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([4, 3, 3, 3, 3, 3, 3, 3, 2, 3], dtype=int32)

In [21]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

sf_merged = sf_data
sf_merged = sf_merged.merge(neighborhoods_venues_sorted, on = 'Neighborhood')

sf_merged.head()

,Zip Code,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,94102,Hayes Valley/Tenderloin/North of Market,37.780,-122.420,3,Coffee Shop,Cocktail Bar,Performing Arts Venue,Vietnamese Restaurant,Café,French Restaurant,Theater,Beer Bar,Mexican Restaurant,Southern / Soul Food Restaurant
1,94103,South of Market,37.780,-122.410,3,Coffee Shop,Sandwich Place,Theater,American Restaurant,Café,Bakery,Vietnamese Restaurant,Wine Bar,Restaurant,Plaza
2,94107,Potrero Hill,37.770,-122.390,2,Food Truck,Gym,Coffee Shop,Pharmacy,Performing Arts Venue,Pizza Place,Harbor / Marina,Park,Basketball Stadium,Café
3,94108,Chinatown,37.791,-122.409,3,Hotel,Coffee Shop,American Restaurant,Boutique,Men's Store,Cocktail Bar,Church,Jewelry Store,Spa,Café
4,94109,Polk/Russian Hill (Nob Hill),37.790,-122.420,3,Grocery Store,Massage Studio,Sushi Restaurant,Bar,Thai Restaurant,Vietnamese Restaurant,Bakery,Diner,Coffee Shop,Pet Store


#### Visualizing the clusters on a Map using Folium

In [22]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_merged['Latitude'], sf_merged['Longitude'], sf_merged['Neighborhood'], sf_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

#### Examining each Cluster

In [34]:
sf_merged.loc[sf_merged['Cluster Labels'] == 0, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,St. Francis Wood/Miraloma/West Portal,Fountain,Bus Line,Park,Scenic Lookout,Event Space,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Fish Market


In [35]:
sf_merged.loc[sf_merged['Cluster Labels'] == 1, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,Visitacion Valley/Sunnydale,Garden,Park,Baseball Field,Yoga Studio,Event Space,Food & Drink Shop,Food,Fondue Restaurant,Flower Shop,Fish Market


In [36]:
sf_merged.loc[sf_merged['Cluster Labels'] == 2, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Potrero Hill,Food Truck,Gym,Coffee Shop,Pharmacy,Performing Arts Venue,Pizza Place,Harbor / Marina,Park,Basketball Stadium,Café
17,Lake Merced,Gym,Performing Arts Venue,Park,Sandwich Place,Café,Paper / Office Supplies Store,Coffee Shop,Rental Car Location,Cocktail Bar,Fish Market


In [38]:
sf_merged.loc[sf_merged['Cluster Labels'] == 3, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Hayes Valley/Tenderloin/North of Market,Coffee Shop,Cocktail Bar,Performing Arts Venue,Vietnamese Restaurant,Café,French Restaurant,Theater,Beer Bar,Mexican Restaurant,Southern / Soul Food Restaurant
1,South of Market,Coffee Shop,Sandwich Place,Theater,American Restaurant,Café,Bakery,Vietnamese Restaurant,Wine Bar,Restaurant,Plaza
3,Chinatown,Hotel,Coffee Shop,American Restaurant,Boutique,Men's Store,Cocktail Bar,Church,Jewelry Store,Spa,Café
4,Polk/Russian Hill (Nob Hill),Grocery Store,Massage Studio,Sushi Restaurant,Bar,Thai Restaurant,Vietnamese Restaurant,Bakery,Diner,Coffee Shop,Pet Store
5,Inner Mission/Bernal Heights,Mexican Restaurant,Grocery Store,Dive Bar,Italian Restaurant,Bakery,Cocktail Bar,Gym / Fitness Center,Pizza Place,Massage Studio,Coffee Shop
6,Ingelside-Excelsior/Crocker-Amazon,Pizza Place,Mexican Restaurant,Chinese Restaurant,Vietnamese Restaurant,Coffee Shop,Café,Sandwich Place,Bar,Pharmacy,Gas Station
7,Castro/Noe Valley,Gay Bar,Park,Yoga Studio,Grocery Store,Thai Restaurant,Clothing Store,Coffee Shop,Playground,Wine Bar,Pet Store
8,Western Addition/Japantown,Park,Yoga Studio,Sandwich Place,Sushi Restaurant,Chinese Restaurant,Furniture / Home Store,Spa,Historic Site,Salon / Barbershop,Playground
9,Parkside/Forest Hill,Chinese Restaurant,Park,Pizza Place,Bubble Tea Shop,Light Rail Station,Sandwich Place,Café,Shoe Repair,Breakfast Spot,Burrito Place
10,Haight-Ashbury,Coffee Shop,Tennis Court,Grocery Store,Yoga Studio,Dog Run,Boutique,Bookstore,Restaurant,Gastropub,Comic Shop


In [39]:
sf_merged.loc[sf_merged['Cluster Labels'] == 4, sf_merged.columns[[1] + list(range(5, sf_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Bayview-Hunters Point,Food & Drink Shop,Motorcycle Shop,Park,Coffee Shop,Restaurant,Ethiopian Restaurant,Food,Fondue Restaurant,Flower Shop,Fish Market


####   
## As we can see from the map and the tables, Cluster 3 is the largest, followed by Cluster 2. Neighbourhoods in these clusters have similar venues and features. This will help in further analysis.
##     

#### I now make a new dataframe with only the venue categories mentioned earlier that are of interest to us for our analysis

In [53]:
sf_health = sf_grouped[['Neighborhood','Organic Grocery','Farmers Market','Vegetarian / Vegan Restaurant']]
sf_health

,Neighborhood,Organic Grocery,Farmers Market,Vegetarian / Vegan Restaurant
0,Bayview-Hunters Point,0.000000,0.000000,0.00
1,Castro/Noe Valley,0.000000,0.000000,0.00
2,Chinatown,0.000000,0.000000,0.00
3,Haight-Ashbury,0.000000,0.000000,0.00
4,Hayes Valley/Tenderloin/North of Market,0.000000,0.010000,0.02
5,Ingelside-Excelsior/Crocker-Amazon,0.000000,0.000000,0.00
6,Inner Mission/Bernal Heights,0.000000,0.000000,0.00
7,Inner Richmond,0.000000,0.015385,0.00
8,Lake Merced,0.000000,0.000000,0.00
9,Marina,0.000000,0.000000,0.00


#### We Cluster this data into 5 clusters to compare with the overall neighbourhood clusters and try to gain some insights

In [54]:
sfclusters = 5

sf_clustering = sf_health.drop(["Neighborhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=sfclusters, random_state=1)
kmeans.fit_transform(sf_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20]

array([0, 0, 0, 0, 1, 0, 0, 4, 0, 0, 0, 0, 0, 3, 2, 0, 0, 0, 0, 0],
      dtype=int32)

In [55]:
sf_health_merged = sf_health.copy()

# add clustering labels
sf_health_merged["Cluster Labels"] = kmeans.labels_


In [56]:
sf_health_merged = sf_health_merged.merge(sf_venues.set_index("Neighborhood"), on="Neighborhood")

print(sf_health_merged.shape)
sf_health_merged.head()

(1086, 11)


,Neighborhood,Organic Grocery,Farmers Market,Vegetarian / Vegan Restaurant,Cluster Labels,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bayview-Hunters Point,0.0,0.0,0.0,0,37.73,-122.38,Hilltop Park,37.732121,-122.383980,Park
1,Bayview-Hunters Point,0.0,0.0,0.0,0,37.73,-122.38,MotoTireGuy - Motorcycle Tire Services,37.726075,-122.380314,Motorcycle Shop
2,Bayview-Hunters Point,0.0,0.0,0.0,0,37.73,-122.38,Ol' McDonald 's Honey Farms & Bee Keepin School,37.726762,-122.382852,Food & Drink Shop
3,Bayview-Hunters Point,0.0,0.0,0.0,0,37.73,-122.38,Cafe Alma,37.731968,-122.375441,Coffee Shop
4,Bayview-Hunters Point,0.0,0.0,0.0,0,37.73,-122.38,Crêpe & Brioche Inc.,37.725685,-122.379370,Restaurant


In [59]:
# create map
map_clusters = folium.Map(location = [latitude, longitude], zoom_start = 12)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i * x) ** 2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(sf_health_merged['Neighborhood Latitude'], sf_health_merged['Neighborhood Longitude'], sf_health_merged['Neighborhood'], sf_health_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius = 5,
        popup = label,
        color = rainbow[cluster - 1],
        fill = True,
        fill_color = rainbow[cluster - 1],
        fill_opacity = 0.7).add_to(map_clusters)
       
map_clusters

#   __________
#    
# **Analysis**

####  
## As we can see from the map, there are several differences between the clusters generated of the overall neighbourhoods and those generated when focusing on the health food related categories we focused on

### The neighbourhoods of Hayes Valley/Tenderloin/North of Market, South of Market and Inner Richmond are all part of the same biggest cluster, cluster 3 of the neighbourhoods overall, but when evaluated on health food venue categories, they all stand alone as distinct and dissimilar clusters
### Similarly, Lake Merced, which overall is more similar to Potrero Hill, here stands as part of Cluster 0, the largest cluster of neighbourhoods similar in terms heath food venues
### The neighbourhoods of Bayview-Hunters Point, Visitacion Valley/Sunnydale and St. Francis Wood/Miraloma/West Portal are all also part of Cluster 0 here and similar in terms of health food venues, whereas in terms of overall venues and similarity they are all unique